# Pipeline

In [ ]:
import sys

sys.path.append(".../pipeline_imaging/imaging")  #'/path/to/your/project'

In [ ]:
import imaging_utils
import os

#  Folder structures

In [ ]:
# Pooled date folders across three sites
# step1 (pooled)/
#  ├── UW_Spectralis_20240701-20240731/
#  └── UCSD_Spectralis_20240701-20240731/
#      └──DICOM
#             └──MC1
#                 └──3
#                    ...many subfolders...
#                    └──00000030 (this is a dicom file)

# => spectralis_instance.organize =>

# DICOM files orangnized by protocols 6 subfolders
# step2/
#  ├── no_rules_apply/
#  ├── raw_data_storage/
#  ├── secondary_capture/
#  ├── spectralis_mac_20x20_hs_octa_oct/
#  ├── spectralis_mac_20x20_hs_octa_retinal_photography/
#  ├── spectralis_onh_rc_hr_oct/
#  ├── spectralis_onh_rc_hr_retinal_photography/
#  ├── spectralis_ppol_mac_hr_oct/
#  ├── spectralis_onh_rc_hr_retinal_photography/
#  ├── spectralis_ppol_mac_hr_oct_small/
#  ├── spectralis_ppol_mac_hr_retinal_photography/
#  └── spectralis_ppol_mac_hr_retinal_photography_small/
#      ├── file1.dcm
#      ├── file2.dcm
#      └── ... (additional DICOM files)

#  => spectralis_instance.convert =>

# DICOM files are formatted to be NEMA compliant, still organized by protocols
# step3/
#     ├── spectralis_onh_rc_hr_oct/
#     ├── spectralis_onh_rc_hr_retinal_photography/
#     ├── spectralis_ppol_mac_hr_oct/
#     ├── spectralis_onh_rc_hr_retinal_photography/
#     ├── spectralis_ppol_mac_hr_oct_small/
#     ├── spectralis_ppol_mac_hr_retinal_photography/
#     └── spectralis_ppol_mac_hr_retinal_photography_small/
#          ├── file1.dcm
#          ├── file2.dcm
#          └── ... (additional DICOM files)

#  => imaging_utils.format_file =>
#  => spectralis_instance.metadata =>

# De-identified again, file renamed, and organized in a final structure for data relase
# Output is saved in two places: step4/ (final data) and step_metadata/ (metadata)
# step4/
#    ├── retinal_photography/
#         ├── ir/
#              ├── heidelberg_spectralis
#                  ├── 1001
#                  ├── 1002
#                  ├── 1003
#                       └── file1.dcm
#                       └── file2.dcm

#    ├── retinal_oct/
#         ├── structural_oct/
#                ├── heidelberg_spectralis
#                    ├── 1001
#                    ├── 1002
#                    ├── 1003
#                         └── file1.dcm
#                         └── file2.dcm


# step_metadata/
#    ├── filename1.json
#    ├── filename2.json
#    ├── filename3.json
#    └── ... (additional JSON files)

# Spectralis instance

In [ ]:
from imaging_spectralis_root import Spectralis

spectralis_instance = Spectralis()

# Organize Spectralis files by protocol


In [ ]:
# working

batch_folder = ".../step1/UW_Spectralis_20240708-20240716"
output_folder = ".../step2"

filtered_list = imaging_utils.spectralis_get_filtered_file_names(batch_folder)


for file in filtered_list:
    organize_result = spectralis_instance.organize(file, output_folder)

# Convert DICOM files to NEMA compliant DICOM files

In [ ]:
# These are fixed options for protocols
protocols = [
    "spectralis_onh_rc_hr_oct",
    "spectralis_onh_rc_hr_retinal_photography",
    "spectralis_ppol_mac_hr_oct",
    "spectralis_ppol_mac_hr_oct_small",
    "spectralis_ppol_mac_hr_retinal_photography",
    "spectralis_ppol_mac_hr_retinal_photography_small",
]

for protocol in protocols:
    output = f".../step3/{protocol}"
    if not os.path.exists(output):
        os.makedirs(output)

    files = imaging_utils.get_filtered_file_names(f".../step2/{protocol}")

    for file in files:
        spectralis_instance.convert(file, output)

# (1) Files are de-identified again, file renamed, and organized in a final structure for data relase (2) Save Metadata for Manifest in json files 

In [ ]:
input_folder = ".../step3"
output_folder = ".../step4"
metadata_folder = ".../step_metadata"


for folder in [input_folder]:
    filelist = imaging_utils.get_filtered_file_names(folder)

    for file in filelist:
        full_file_path = imaging_utils.format_file(file, output_folder)
        spectralis_instance.metadata(full_file_path, metadata_folder)